# Host Filtering Assessment

Filtering the barley genome from NT metagenomic reads was observed to bias taxonomic classifications, with a reduction in Firmicutes. Let's try to work out why...

## Barley Genome

We can use the Morex V3 assembly which includes PacBio HiFi reads, and so is less likely to have bacterial contaminants in the assembly than Morex V2 which was used for the previous analysis. 

In [3]:
import urllib3
import shutil
import subprocess
import os
import re
from glob import glob

In [14]:
os.mkdir('reference')
http = urllib3.PoolManager()

fasta_url='https://doi.ipk-gatersleben.de/DOI/b2f47dfb-47ff-4114-89ae-bad8dcc515a1/b6e6a2e5-2746-4522-8465-019c8f56df7f/1/DOWNLOAD'
annot_url='https://doi.ipk-gatersleben.de/DOI/b2f47dfb-47ff-4114-89ae-bad8dcc515a1/5d16cc17-c37f-417f-855d-c5e72c721f6c/1/DOWNLOAD'
fasta_path='reference/Barley_MorexV3_psuedomolecules.fasta'
annot_path='reference/Barley_MorexV3_psuedomolecules.gff3'
    
print('Downloading {}'.format(fasta_url))
with http.request('GET',fasta_url, preload_content=False) as resp, open(fasta_path, 'wb') as out_file:
    shutil.copyfileobj(resp, out_file)
r.release_conn()

print('Downloading {}'.format(annot_url))
with http.request('GET',annot_url, preload_content=False) as resp, open(annot_path, 'wb') as out_file:
    shutil.copyfileobj(resp, out_file)
r.release_conn()

Now we can build a BWA index...

In [19]:
os.mkdir('bwa')
os.mkdir('logs')
bwa_path='bwa/MorexV3'
os.symlink('../'+fasta_path,bwa_path)

process=subprocess.Popen(['bwa','index',bwa_path],
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE)
stdout,stderr = process.communicate()

with open('logs/bwa_morex_v3_index.out','w') as stdout_file:
    stdout_file.write(stdout.decode('UTF-8'))

with open('logs/bwa_morex_v3_index.err','w') as stderr_file:
    stderr_file.write(stderr.decode('UTF-8'))

...and symlink in the original QC'd fastq files...

In [15]:
files=glob('../GO_enrichment/read_qc/trim_galore/*gz')
if not os.path.exists('fastq'):
    os.mkdir('fastq')

fq_re=re.compile(r'([0-9]{4}_[12])_val_[12].fq.gz')
for file in files:
    m=re.search(fq_re,file)
    os.symlink('../'+file,'fastq/{}.fq.gz'.format(m.group(1)))

In [17]:
%%bash
cat bin/align_and_filter.sh
#qsub bin/align_and_filter

UsageError: Cell magic `%%bashcat` not found.
